In [9]:
import pandas as pd
import pandas_ta as ta
pd.set_option('display.max_rows', 500)
import requests,json
import numpy as np

In [10]:
# Create dict with symbols I bough
# Get hour data from kucoin
# Get latest price available

In [11]:
kucoin_position = {
    'VISION-USDT': {
        'price': 0.0081531,
        'buy_date' : '2023-12-04',
        'quantity': 10306.7021,
        },

    'POLS-USDT': {
        'price': 0.4419,
        'buy_date' : '2023-12-03',
        'quantity': 129.4073,
        },

    'SUPER-USDT': {
        'price': 0.4553,
        'buy_date' : '2023-12-03',
        'quantity': 127.5088,
        'sold_date' : '2023-12-04',
        'sold_price': 0.65657,
        }
}

## Sold Cases

In [17]:
df_control = pd.DataFrame(kucoin_position).T.reset_index().sort_values(by=['buy_date'])
df_control['profit'] = df_control['sold_price'] * df_control['quantity'] - df_control['price'] * df_control['quantity']

## Open Cases

In [13]:
df_open = df_control[~df_control['sold_date'].notnull()]

In [14]:
def getting_data(symbol):
    print('Getting ' + symbol)

    kucoin = requests.get(f'https://api.kucoin.com/api/v1/market/orderbook/level1?symbol={symbol}')
    if kucoin.status_code == 200:  # if result is GOOD
        data = json.loads(kucoin.text)['data']  # load the json response from API
        df = pd.DataFrame([data])  # create dataframe out of json response
        df['price'] = df['price'].map(lambda x: float(x))
        df['symbol'] = f'{symbol.replace("-", "")}'
        df['time_period_start'] = pd.to_datetime(df['time'].astype(int), unit='ms')
        df.sort_values(by='time_period_start', inplace = True)
        df.drop(columns=['sequence', 'size', 'bestBid', 'bestBidSize', 'bestAsk', 'bestAskSize', 'time'], inplace=True)
    else:
        pass

    return df

In [15]:
testing = []
for index, row in df_open.iterrows():
    crypto = row['index']
    price = row['price']
    try:
        data = getting_data(crypto)
        if not data.empty:  # Check if the dataframe is not empty before appending
            data['price_bought'] = price
            data['profit_loss'] = (((data['price'] / data['price_bought']) - 1) * 100).round(2).astype(float)
            testing.append(data)
    except:
        pass

if testing:  # Check if the list is not empty before concatenating
    df_concat = pd.concat(testing, ignore_index=True)
else:
    # Handle the case when no data is available
    df_concat = pd.DataFrame()

Getting VISION-USDT
Getting POLS-USDT


In [16]:
df_concat

,price,symbol,time_period_start,price_bought,profit_loss
0,0.008164,VISIONUSDT,2023-12-04 19:42:41.872,0.008153,0.13
1,0.471000,POLSUSDT,2023-12-04 19:41:34.766,0.441900,6.59
